In [4]:
from fuzzywuzzy import fuzz as fw_fuzz, process as fw_process
from rapidfuzz import fuzz as rapid_fuzz, process as rapid_process
import Levenshtein
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from functools import wraps
import ipywidgets as widgets
import requests

In [5]:
def print_timing(f):
    @wraps(f)
    def inner(*args, **kwargs):
        t0 = timer()
        try:
            return f(*args, **kwargs)
        finally:
            print(f"\n{f.__name__} finished in {1e3*(timer()-t0):.1f} ms")
    return inner

In [6]:
all_companies_content = requests.get("https://www.sec.gov/Archives/edgar/cik-lookup-data.txt").content.decode("latin1")
all_companies_array = all_companies_content.split("\n")

# remove records that can't be converted to dictionary (has : in companyname)
l=len(all_companies_array)-1
for i in reversed(all_companies_array):
    semicol = i.count(":")
    if semicol != 2:
        # print(all_companies_array[l],"has",semicol,"delimiters! Deleting...")
        del all_companies_array[l]
    l=l-1

all_companies_arra = [i[:-1] for i in all_companies_array] #remove trailing :
all_companies_cik_dict=dict(item.split(":") for item in all_companies_arra)

In [7]:
@print_timing
def rapid_companyName(input_text):
    if not input_text: return
    for company, match_score, _ in rapid_process.extract(input_text, all_companies_arra, limit=10):
        print(f"{company:60}{match_score:3.2f}")

widgets.interact(rapid_companyName, input_text="")

interactive(children=(Text(value='', description='input_text'), Output()), _dom_classes=('widget-interact',))

<function __main__.interact_companyName(input_text)>

In [10]:
@print_timing
def fuzzywozzy_companyName(input_text):
    if not input_text: return
    for company, match_score in fw_process.extract(input_text, all_companies_arra, limit=10):
        print(f"{company:60}{match_score:3.2f}")

widgets.interact(fuzzywozzy_companyName, input_text="")

interactive(children=(Text(value='', description='input_text'), Output()), _dom_classes=('widget-interact',))

<function __main__.fuzzywozzy_companyName(input_text)>